In [1]:
import numpy as np
import pandas as pd
import os
import re


In [2]:
def funcion1(path = 'data/train-set-to-publish/cantemist-ner/'):
    
    """Esta función sirve para leer los archivos de anotaciones y documentos contenidos en path/cantemist-ner/. 
    La entrada de esta función es un directorio con documentos .ann y .txt.
    La salida son dos diccionarios, uno con anotaciones y otro con documentos de texto."""
    
    
    ANNOTdict = {}
    CORPUSdict = {}
    
    filelist = os.listdir(path)

    for x in filelist:
        
        try:

            if x[-3:] == 'ann':
                tempdf = pd.read_csv(path+x,sep="\t",encoding='UTF8',)
                templabel = x[:-4]

                firstrow = pd.Series(pd.read_csv(path+x,sep="\t",encoding='UTF8',).columns,
                                     index=['T','MORFOLOGIA','CLAVE'])

                d = {}

                for y in enumerate(firstrow):

                    d[y[1]] = firstrow.index[y[0]]

                tempdf.rename(d,axis=1,inplace=True)

                tempdf = tempdf.append(firstrow,ignore_index=True)

                tempdf['T'].replace(
                    {"T1":"T01","T2":"T02","T3":"T03","T4":"T04","T5":"T05",
                     "T6":"T06","T7":"T07","T8":"T08","T9":"T09",},
                    inplace=True,)

                tempdf = tempdf.sort_values(by='T').reset_index(drop=True)

                ANNOTdict[templabel] = tempdf

            elif x[-3:] == 'txt':
                tempdf = pd.Series(data = open(path+x,encoding='UTF8').read())
                templabel = x[:-4]
                CORPUSdict[templabel] = tempdf
        
        except Exception as e: 
            print(f'{e} {path + x}')
        
    return (ANNOTdict,CORPUSdict)



In [19]:
def funcion2(ANNOTdict,CORPUSdict):
    
    
    """Esta función sirve para indexar la posición de las palabras contenidas en los archivos .ann dentro de su respectivo .txt.
    La entrada de esta función son dos diccionarios, uno de anotaciones y otro de documentos.
    La salida son dos diccionarios, el primero contiene la posición de las palabras indexada, el segundo contiene las anotaciones junto con la posición."""
    
    
    pos = {}


    for x in ANNOTdict.keys():
        pos[x] = pd.DataFrame(columns=['startposition','endposition'])

        templiststart = []

        templistend = []

        for y in enumerate(ANNOTdict[x]['CLAVE']):

            start = CORPUSdict[x][0].find(ANNOTdict[x]['CLAVE'][y[0]])
            end = CORPUSdict[x][0].find(ANNOTdict[x]['CLAVE'][y[0]]) + len(y[1])

            templiststart.append(start)
            templistend.append(end)

        pos[x]['startposition'] = templiststart
        pos[x]['endposition'] = templistend

        ANNOTdict[x] = ANNOTdict[x].join(pos[x])
    
    return pos,ANNOTdict

In [4]:
def funcion3(path = 'data/train-set-to-publish/cantemist-norm/'):

    """Esta función sirve para leer los archivos de anotaciones y documentos contenidos en path/cantemist-norm/. 
    La entrada de esta función es un directorio con documentos .ann y .txt.
    La salida son dos diccionarios, uno con anotaciones y otro con documentos de texto."""
    
    
    ANNOTdict = {}
    CORPUSdict = {}

    filelist = os.listdir(path)

    for x in filelist:

        try:
            if x[-3:] == 'ann':
                tempdf = pd.read_csv(path+x,sep="\t",encoding='UTF8',)
                templabel = x[:-4]

                firstrow = pd.Series(pd.read_csv(path+x,sep="\t",encoding='UTF8',).columns,
                                     index=['T','MORFOLOGIA','CLAVE'])

                d = {}

                for y in enumerate(firstrow):

                    d[y[1]] = firstrow.index[y[0]]

                tempdf.rename(d,axis=1,inplace=True)

                tempdf = tempdf.append(firstrow,ignore_index=True)

                tempdf['T'].replace(
                    {"#":"U"},
                    inplace=True,regex=True)

                tempdf['T'].replace(
                    {"T1":"T01","T2":"T02","T3":"T03","T4":"T04","T5":"T05",
                     "T6":"T06","T7":"T07","T8":"T08","T9":"T09",},
                    inplace=True,)

                tempdf['T'].replace(
                    {"U1":"U01","U2":"U02","U3":"U03","U4":"U04","U5":"U05",
                     "U6":"U06","U7":"U07","U8":"U08","U9":"U09",},
                    inplace=True,)

                tempdf = tempdf.sort_values(by='T').reset_index(drop=True)

                ICDcode = tempdf[tempdf['T'].apply(lambda x: x[0]=='U')].rename(
                    {'CLAVE':'ICD-O-3'},axis=1).reset_index(drop=True)['ICD-O-3']

                tempdf = tempdf[tempdf['T'].apply(lambda x: x[0]=='T')].join(ICDcode,)


                ANNOTdict[templabel] = tempdf

                
        except Exception as e: 
                print(f'{e} {path + x}')
                
    return (ANNOTdict,CORPUSdict)
    




In [20]:
def funcion4(dict1=funcion2(funcion1()[0],funcion1()[1])[1],
             dict2=funcion3()[0]):
    
    
    """Esta función sirve para . 
    La primer entrada es un diccionario de archivos .ann que contiene la posición de palabras a indexar.
    La segunda entrada es un diccionario de archivos .ann que contiene el código ICD-O-3.
    La es un diccionario que contiene posición y código ICD-O-3."""
    
    
    tempdict = {}

    for x in dict1.keys():
                
        tempdict[x] = dict1[x].join(dict2[x]['ICD-O-3'])
    
    return tempdict

No columns to parse from file data/train-set-to-publish/cantemist-ner/cc_onco853.ann
No columns to parse from file data/train-set-to-publish/cantemist-ner/cc_onco853.ann
No columns to parse from file data/train-set-to-publish/cantemist-norm/cc_onco853.ann


In [21]:
def funcion5(dict1=funcion4()):
    
    """Regresa una tabla que contiene una morfología encontrada, posición de inicio y de fin, 
    ICD-O-3 y documento en donde se encontró"""
    
    final_df = pd.DataFrame()
    tempdf = pd.DataFrame()

    for key in dict1:
        tempdf = dict1[key]
        tempdf['document'] = key

        final_df = final_df.append(tempdf)
        
    return final_df

In [156]:
def funcion6(trainset = funcion5(), CORPUSdict = funcion1('data/test-background-set-to-publish/')[1],):
    
    """Busca las morfologias contenidas en el set de entrenamiento, dentro de los documentos contenidos en el set de prueba. 
    El set de entrenamiento tiene  filas.
    La salida es un diccionario con las morfologias encontradas, ICD-O-3, documento, índice dentro del documento."""


    tempdf = trainset[['CLAVE','ICD-O-3']][trainset['CLAVE'].apply(lambda x: x.lower()).duplicated() == False]

    tempdf['len'] = tempdf['CLAVE'].apply(lambda x: len(x))
    
    finaldf = pd.DataFrame()

    tempdict = {}
    
    for key in CORPUSdict.keys():

        tempdict[key] = pd.DataFrame()
        
        text = CORPUSdict[key][0]

        for word,code in zip(tempdf['CLAVE'],tempdf['ICD-O-3']):
            
            if len(word)>4:

                startindex = text.lower().find(word.lower())

                if startindex != -1:
                    
                    endindex = startindex+len(word)
                    
                    tempseries = pd.Series(data={'CLAVE':word,'startpos':startindex,'endpos':endindex,'ICD-O-3':code,'documento':key})

                    finaldf = finaldf.append(tempseries,ignore_index=True,)
                
            elif len(word)<=4:
                
                startindex = text.find(word)
                        
                if startindex != -1:
                    
                    endindex = startindex+len(word)
                    
                    if(text[startindex-1] == ' ') or (text[startindex-1] == '\n') or (text[startindex-1] == '.') or (text[startindex-1] == '-') or (text[startindex-1] == ':') or (text[startindex-1] == ';') or (text[startindex-1] == '(') or (text[startindex-1] == ')') or (text[startindex-1] == ',') : 
                    
                        try:
                    
                            if (text[endindex+1] == ' ') or (text[endindex+1] == '\n') or (text[endindex+1] == '.') or (text[endindex+1] == '-') or (text[endindex+1] == ':') or (text[endindex+1] == ';') or (text[startindex-1] == '(') or (text[startindex-1] == ')') or (text[startindex-1] == ',') :

                                tempseries = pd.Series(data={'CLAVE':word,'startpos':startindex,'endpos':endindex,'ICD-O-3':code,'documento':key})

                                finaldf = finaldf.append(tempseries,ignore_index=True,)
                                
                        except:
                            
                            if (len(text) == endindex):
                                
                                tempseries = pd.Series(data={'CLAVE':word,'startpos':startindex,'endpos':endindex,'ICD-O-3':code,'documento':key})

                                finaldf = finaldf.append(tempseries,ignore_index=True,)


    d1 = {}
    for k in finaldf['documento'].unique():
        d1[k] = finaldf[finaldf['documento']==k]
        
    for key in d1.keys():
        
        d1[key]['len'] = d1[key]['CLAVE'].apply(lambda x: len(x))
        
        d1[key] = d1[key].sort_values(by = ['len'],ascending=False)[d1[key].sort_values(by = ['len'],ascending=False)['startpos'].duplicated() == False].sort_values('startpos').reset_index(drop=True)
        
        d1[key] = d1[key].sort_values(by = ['len'],ascending=False)[d1[key].sort_values(by = ['len'],ascending=False)['endpos'].duplicated() == False].sort_values('startpos').reset_index(drop=True)
        
        d1[key].index = map(lambda x: "T"+str(x+1), d1[key].index)
        
        d1[key].drop('len',axis=1,inplace=True)
 
        
    return (d1,tempdf)

In [157]:
d2 = funcion6()

C:\Users\L03040999\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [158]:
len(d2[0].keys())

2809

In [149]:
d2[1]

,CLAVE,ICD-O-3,len
0,Carcinoma microcítico,8041/3,21
2,M0,8000/6,2
0,metastásica,8000/6,11
1,metástasis,8000/6,10
3,neoplasia,8000/1,9
...,...,...,...
21,implante rectovesical,8000/6,21
22,implantes en cara anterior de recto,8000/6,35
23,implante en mesocolon,8000/6,21
2,carcinoma urotelial de vejiga de alto grado,8120/3,43


In [153]:
# def funcion7(d1=funcion6()[0])):
def funcion7(d1,traindf): 
    
    """Regresa las morfologias encontradas en formato NERbrat y NORMbrat."""
    
    NERbratformat = {}

    for key in d1.keys():

        d1[key]['MORFOLOGIA NEOPLASIA'] = 'MORFOLOGIA_NEOPLASIA'

        NERbratformat[key] = d1[key][['MORFOLOGIA NEOPLASIA','startpos','endpos','CLAVE']]
        
        NERbratformat[key][['startpos','endpos']] = NERbratformat[key][['startpos','endpos']].astype(int)
        
        NERbratformat[key].to_csv(f'data/team_bigbyte/cantemist-ner/{key}.ann', header=False, index=True, sep='\t',)
        
    NORMbratformat = {}

    
    for key in NERbratformat.keys():

        NORMbratformat[key] = pd.DataFrame()
        
        NERbratformat[key][['startpos','endpos']] = NERbratformat[key][['startpos','endpos']].astype(int)

        for i,row in enumerate(NERbratformat[key].index):
            tempseries = NERbratformat[key].loc[row].append(pd.Series(data={'index':row},name=row))

            tempseries2 = pd.Series(data = {
                'index':'#'+str(i+1), 'MORFOLOGIA NEOPLASIA':f'AnnotatorNotes {row}','startpos':d1[key].iloc[i]['ICD-O-3']}, 
                                    name=row
                                   )
            
            tempdf = pd.concat([tempseries,tempseries2],axis=1).T.set_index('index').reset_index()

            NORMbratformat[key] = NORMbratformat[key].append(tempdf)
            
        NORMbratformat[key].to_csv(f'data/team_bigbyte/cantemist-norm/{key}.ann', header=False, index=False, sep='\t', )
        
    
    tempdf = pd.DataFrame()

    for x in d1.keys():
    
        tempdf = tempdf.append(d1[x].sort_values(by='startpos')[['documento','ICD-O-3']])
        
    
    tempdf.to_csv(f'data/team_bigbyte/cantemist-coding/coding.tsv', header=False, index=False, sep='\t', )
    traindf.sort_values(by='CLAVE').to_csv(f'data/team_bigbyte/traindf.txt', header=True, index=False, sep='\t', )


In [154]:
funcion7(d2[0],d2[1])

C:\Users\L03040999\Anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [155]:
d2[1].sort_values(by='CLAVE')

,CLAVE,ICD-O-3,len
10,(LOE ) frontal,8000/6,14
18,(LOE) de 15 mm hepática,8000/6,23
11,(LOE) en el hígado,8000/6,18
9,(LOE) hepática,8000/6,14
13,(LOE) hepáticas,8000/6,15
...,...,...,...
11,ypT3N0/M0,8000/6,9
12,ypT3N0M0,8000/6,8
14,ypT3N1M0,8000/6,8
13,ypT3cN1M0,8000/6,9


In [159]:
d2[0]

{'caso_clinico_radiologia1':         CLAVE ICD-O-3                 documento  endpos  startpos
 T1  formación  8000/1  caso_clinico_radiologia1   564.0     555.0,
 'caso_clinico_radiologia10':                        CLAVE ICD-O-3                  documento  endpos  \
 T1       masas suprarrenales  8000/6  caso_clinico_radiologia10  1167.0   
 T2                 neoplasia  8000/1  caso_clinico_radiologia10  1253.0   
 T3                 Carcinoma  8010/3  caso_clinico_radiologia10  1283.0   
 T4                 poliposis  8210/0  caso_clinico_radiologia10  1331.0   
 T5  Carcinoma indiferenciado  8020/3  caso_clinico_radiologia10  1420.0   
 T6                metástasis  8000/6  caso_clinico_radiologia10  1459.0   
 
     startpos  
 T1    1148.0  
 T2    1244.0  
 T3    1274.0  
 T4    1322.0  
 T5    1396.0  
 T6    1449.0  ,
 'caso_clinico_radiologia1000':       CLAVE ICD-O-3                    documento  endpos  startpos
 T1  linfoma  9590/3  caso_clinico_radiologia1000   285.0     